In [2]:
import os
import shutil

src = '../data/raw/'
dst = '../data/clean'

os.makedirs(f'{dst}/0', exist_ok=True)
os.makedirs(f'{dst}/1', exist_ok=True)

for patient_dir in os.listdir(src):
    patient_path = os.path.join(src, patient_dir)
    if not os.path.isdir(patient_path): continue

    for filename in os.listdir(patient_path):
        if filename.endswith('.png'):
            label = filename.split('_')[0]
            src_file = os.path.join(patient_path, filename)
            dst_file = os.path.join(dst, label, filename)
            shutil.copy2(src_file, dst_file)